# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '3b4952a6-80cf-4998-85c9-641235c25488', 'created_at': '2025-08-14T00:55:28.729995+00:00', 'updated_at': '2025-08-14T00:55:28.729995+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
3b4952a6-80cf-4998-85c9-641235c25488


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "한국 반도체 현황. 모두 한글로 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
한국 반도체 산업 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업의 역사 - 한국 반도체 산업의 기원과 발전 과정을 통해, 세계적 경쟁력을 갖추게 된 배경을 살펴봅니다. 1960년대 초반부터 시작된 한국 반도체 산업의 역사를 설명합니다. 
3. 한국 반도체 산업 시장 점유율 분석 - 한국 반도체 산업의 시장 점유율을 다른 국가들과 비교 분석하고, 메모리 및 비메모리 반도체 시장의 상황을 알아봅니다. 주요 경쟁국과의 경쟁력 차이 및 주요 실적 변화를 다룹니다. 
3. 한국 반도체 산업의 경쟁력 분석 - 한국 반도체 산업이 글로벌 시장에서 보유한 기술적, 경제적 경쟁력에 대해 분석합니다. 특히, 메모리 반도체와 시스템 반도체 분야에서의 강점과 약점을 다룹니다. 
4. 한국 반도체 산업 주요 과제 - 한국 반도체 산업이 직면한 도전 과제와 이를 해결하기 위한 정부와 기업의 전략 및 노력을 탐구합니다. 국제 무역 분쟁, 기술의 발전과 혁신의 필요성 등을 다룹니다. 
3. 한국 반도체 산업의 발전 전망 - 향후 한국 반도체 산업의 성장 가능성에 대해 논의하고, 글로벌 시장 내에서 대한민국 반도체 산업의 위치 및 예상되는 미래 발전 방향과 전략을 살펴봅니다. 
5. 결론 및 한국 반도체 산업 발전을 위한 제안 - 한국 반도체 산업의 현황과 미래를 바탕으로 발전을 위한 효율적인 전략과 요구사항을 정리하고 정책적 제안을 제시합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  A

In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
한국 반도체 산업 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업의 역사 - 한국 반도체 산업의 기원과 발전 과정을 통해, 세계적 경쟁력을 갖추게 된 배경을 살펴봅니다. 1960년대 초반부터 시작된 한국 반도체 산업의 역사를 설명합니다. 
3. 한국 반도체 산업 시장 점유율 분석 - 한국 반도체 산업의 시장 점유율을 다른 국가들과 비교 분석하고, 메모리 및 비메모리 반도체 시장의 상황을 알아봅니다. 주요 경쟁국과의 경쟁력 차이 및 주요 실적 변화를 다룹니다. 
3. 한국 반도체 산업의 경쟁력 분석 - 한국 반도체 산업이 글로벌 시장에서 보유한 기술적, 경제적 경쟁력에 대해 분석합니다. 특히, 메모리 반도체와 시스템 반도체 분야에서의 강점과 약점을 다룹니다. 
4. 한국 반도체 산업 주요 과제 - 한국 반도체 산업이 직면한 도전 과제와 이를 해결하기 위한 정부와 기업의 전략 및 노력을 탐구합니다. 국제 무역 분쟁, 기술의 발전과 혁신의 필요성 등을 다룹니다. 
3. 한국 반도체 산업의 발전 전망 - 향후 한국 반도체 산업의 성장 가능성에 대해 논의하고, 글로벌 시장 내에서 대한민국 반도체 산업의 위치 및 예상되는 미래 발전 방향과 전략을 살펴봅니다. 
5. 결론 및 한국 반도체 산업 발전을 위한 제안 - 한국 반도체 산업의 현황과 미래를 바탕으로 발전을 위한 효율적인 전략과 요구사항을 정리하고 정책적 제안을 제시합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  A

In [9]:
len(state["values"]["slides"])

8

In [10]:
state["values"]["slides"]

[{'description': '한국 반도체 산업 현황에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'slide_type': 'title',
  'topic': '한국 반도체 산업 현황',
  'idx': 0,
  'name': '타이틀',
  'content': '## 타이틀: 한국 반도체 산업 현황 2023\n\n한국은 세계 반도체 산업에서 중요한 위치를 차지하고 있습니다. 반도체는 한국 최대 수출 품목 중 하나로 2022년 총 수출액의 18.9%를 차지했습니다. 2023년에는 반도체 수출이 전년 대비 감소세를 보였지만, 메모리 반도체의 가격 하락이 주요 원인으로 작용했습니다. 한국은 특히 메모리 반도체 분야에서 세계시장을 선도하고 있으며, 삼성전자와 SK하이닉스는 이 시장을 이끄는 주요 기업입니다. 반면, 비메모리 반도체 분야에서는 글로벌 시장 점유율 6위를 기록하고 있습니다. 한국 반도체 산업은 기술 개발과 시장 확대를 통해 앞으로도 높은 경쟁력을 유지하려고 하고 있으며, 반도체가 한국 경제에 차지하는 비중과 중요성은 계속 성장할 것으로 기대됩니다.\n\n1. [KDI 경제교육·정보센터](https://eiec.kdi.re.kr/policy/domesticView.do?ac=0000172912&issus=)\n2. [Invest Korea](https://www.investkorea.org/ik-en/bbs/i-308/detail.do?ntt_sn=490787)\n3. [한국경제인협회](https://www.fki.or.kr/webzine/2412/sub_30.html)\n4. [전기신문](https://www.electimes.com/news/articleView.html?idxno=356183)\n5. [PwC](https://www.pwc.com/kr/ko/insights/industry-focus/samilpwc_semiconductor

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])